In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# install dependencies:
!pip install pyyaml==5.1 pycocotools>=2.0.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

# install detectron2: (Colab has CUDA 10.1 + torch 1.6)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
assert torch.__version__.startswith("1.6")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

1.6.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html


In [3]:
import sys, getopt
sys.path.append('/content/drive/My Drive/github/object_detection/universal-master/python')
print(sys.path)

from pycocotools.coco import COCO
import cv2, json, os
import numpy as np
from google.colab.patches import cv2_imshow

import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

import tensorflow as tf
from tensorflow.python.platform import gfile
import os.path
import matplotlib.pyplot as plt
from timeit import time

if sys.version_info[0] >= 3:
    from urllib.request import urlretrieve
else:
    from urllib import urlretrieve

# from prepare_imagenet_data import preprocess_image_batch, create_imagenet_npy, undo_image_avg
from universal_pert import universal_perturbation

['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/My Drive/github/object_detection/universal-master/python']


In [13]:
def get_config_file(dataset_name, model_file):
  cfg = get_cfg()
  cfg.merge_from_file(model_zoo.get_config_file(model_file)) # select model of your choice
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(MetadataCatalog.get(dataset_name).thing_classes)  # number of classes 
  
  return cfg

def coco_images(dataDir, dataType):
  """ load images of specific categories in COCO dataset 

  :return: list[image]

  :param dataDir: str, path to the folder where your annotation files for your videos(frames)
  :param dataType: str, dataset split in COCO (train or val)

  getAnnIds: Get ann ids that satisfy given filter conditions. 
  getCatIds: Get cat ids that satisfy given filter conditions. 
  getImgIds: Get img ids that satisfy given filter conditions. 

  loadAnns: Load anns with the specified ids. 
  loadCats: Load cats with the specified ids. 
  loadImgs: Load imgs with the specified ids. 

  loadResLoad: algorithm results and create API for accessing them. 
  showAnnsDisplay: the specified annotations.
  """ 

  annFile='{}/annotations/instances_{}.json'.format(dataDir, dataType)
  imgs = []

  # initialize COCO api for instance annotations
  coco = COCO(annFile)
  # cat_ids = coco.getCatIds()
  # cats = coco.loadCats(cat_ids)
  
  # Define the classes (out of the 81) which you want to get
  filterClasses = ['car']

  # Get class IDs only corresponding to the filterClasses
  catIds = coco.getCatIds(catNms=filterClasses) # finds category['id'] : category['name'] -> find category['id'] matching category['name']

  # Get all image ids containing the above Category IDs
  # finds annotation[image_id] : category['id'] -> find annotation[category_id] matching category['id'] -> find annotation[image_id] matching annotation[category_id]
  imgIds = coco.getImgIds(catIds=catIds) 
  imgIds.sort()
  print("Number of images containing classes {}: {}".format(filterClasses, len(imgIds)))

  # load the selected images     
  for i in range(20):
    print(i)
    img_dict = coco.loadImgs(imgIds[i])[0] # get image dict by using image['id'] (we need image['file_name'] to load images of that image['id'])
    img = cv2.imread('{}/{}/{}'.format(dataDir, dataType, img_dict['file_name']))
    cv2_imshow(img)
    imgs.append(img)
    
  print("FINISHED LOADING IMAGES")
  return imgs


def extract_roi_labels(imgs, dataset_name, model_file, thresh):
  """extract labels and boxes for each instance(roi) from the selected images

  :output: 

  :param imgs: the dataset that you want to test your trained detector on (must be registered via func 'register_datasets')
  :type imgs: str

  :param dataset_name: the dataset that you want to test your trained detector on (must be registered via func 'register_datasets')
  :type dataset_name: str

  :param model_file: the config file for your object detection model that you want to train on
  :type model_file: str

  :param thresh: set a custom testing threshold for predicting as positive
  :type thresh: int
  """ 
  rois = []
  labels = []

  # get dataset and model for prediction
  cfg = get_config_file(dataset_name, model_file)
  cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model_file)
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = thresh  
  
  predictor = DefaultPredictor(cfg)

  for im in imgs:
    outputs = predictor(im)

    # make a prediction on each image and draw predictions on the image
    instances = outputs["instances"]
    category_instances = instances
    
    # for each instance boxes
    box_list = category_instances.pred_boxes
    for inst_box in box_list:

      # crop the instance inside the image as a new image for computing universal perturbation
      roi = im[int(inst_box[1]):int(inst_box[3]), int(inst_box[0]):int(inst_box[2])]
      roi = cv2.resize(roi, (224, 224))
      rois.append(roi)

    # make labels for each instance
    labels.extend(category_instances.pred_classes)
  
  return cfg, predictor, rois, labels

In [ ]:
def jacobian(y_flat, x, inds, num_classes):
    n = num_classes # Not really necessary, just a quick fix.
    loop_vars = [
         tf.constant(0, tf.int32),
         tf.TensorArray(tf.float32, size=n),
    ]
    _, jacobian = tf.while_loop(
        lambda j,_: j < n,
        lambda j,result: (j+1, result.write(j, tf.gradients(y_flat[inds[j]], x))),
        loop_vars)
    
    return jacobian.stack()

def universal_perturb(cfg, X, predictor, label_originals, str_label_original):
    # Default values
    device = '/gpu:0'
    num_classes = cfg.MODEL.ROI_HEADS.NUM_CLASSES
        
    with tf.device(device):
        persisted_sess = tf.Session()
        
        def f(image_inp): 
            outputs = predictor(image_inp)
            return outputs['instances'].pred_classes

        persisted_sess.graph.get_operations()

        persisted_input = persisted_sess.graph.get_tensor_by_name("input:0")
        persisted_output = persisted_sess.graph.get_tensor_by_name("softmax2_pre_activation:0")

        print('>> Computing feedforward function...')
        
        file_perturbation = os.path.join('data', 'universal_coco2017val_car_fasterrcnn.npy')
        
        # if precomputed file doesn't exist, you have to calculate it
        if os.path.isfile(file_perturbation) == 0:

            print('>> Compiling the gradient tensorflow functions. This might take some time...')
            y_flat = tf.reshape(f(image_inp), (-1,))
            inds = tf.placeholder(tf.int32, shape=(num_classes,))
            dydx = jacobian(y_flat, persisted_input, inds, num_classes)

            print('>> Computing gradient function...')
            def grad_fs(image_inp, indices): 
              return persisted_sess.run(dydx, feed_dict={persisted_input: image_inp, inds: indices}).squeeze(axis=1)

            # compute universal perturbation v on image X and function f
            v = universal_perturbation(X, f, grad_fs, delta=0.2, num_classes=num_classes)

            # Save the universal perturbation to your path
            np.save(os.path.join(file_perturbation), v)

        else:
            print('>> Found a pre-computed universal perturbation! Retrieving it from ", file_perturbation')
            v = np.load(file_perturbation)

        return v

        print('>> Testing the universal perturbation on an image')
        image_original = X
        # Test the perturbation on the image
        # Clip the perturbation to make sure images fit in uint8
        clipped_v = np.clip(undo_image_avg(image_original[0,:,:,:]+v[0,:,:,:]), 0, 255) - np.clip(undo_image_avg(image_original[0,:,:,:]), 0, 255)
        image_perturbed = image_original + clipped_v[None, :, :, :]
        label_perturbed = np.argmax(f(image_perturbed), axis=1).flatten()

        print(label_original)
        print(label_perturbed)

In [15]:
if __name__ == '__main__':
    dataDir='/content/drive/My Drive/datasets/coco'
    dataType='val2017'
    
    # 1. use pycocotools lib to extract images of specific categories in COCO
    imgs = coco_images(dataDir, dataType)
    
    dataset_name = 'coco_2017_val'
    conf_file = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
    thresh = 0.7

    # 2. Use Object Detector to extract object instances in those selected images as training images
    cfg, predictor, rois, labels = extract_roi_labels(imgs, dataset_name, conf_file, thresh)
    
    # 3. Use Object Detector to extract object instances in those selected images as training images
    universal_perturb(cfg, rois, predictor, labels, MetadataCatalog.get(dataset_name).thing_classes)

Output hidden; open in https://colab.research.google.com to view.